# Construction complète des données PhysAE

Ce notebook expose des cellules autonomes pour générer des spectres synthétiques, ajuster tous les paramètres physiques et explorer l'impact de chaque choix.

## Imports et constantes

La cellule suivante regroupe toutes les dépendances nécessaires pour manipuler les données et tracer les spectres.

In [ ]:
from pprint import pprint
from copy import deepcopy
from pathlib import Path

import torch
import matplotlib.pyplot as plt

from physae import config
from physae.config_loader import load_data_config, merge_dicts
from physae.dataset import SpectraDataset
from physae.physics import parse_csv_transitions

## Chargement et inspection de la configuration par défaut

On charge la configuration `default.yaml` fournie par PhysAE puis on en extrait les sections utiles pour préparer un jeu de données.

In [ ]:
data_cfg = load_data_config(name="default")
print("Clés disponibles :")
print(sorted(data_cfg.keys()))

print("
Plages d'entraînement (avant expansion) :")
pprint(data_cfg['train_ranges_base'])

print("
Paramètres de bruit (train) :")
pprint(data_cfg['noise']['train'])

## Définition d'un scénario personnalisé

On peut modifier **toutes** les plages de tirage, le profil de bruit et le nombre d'échantillons en effectuant des copies profondes de la configuration initiale.

In [ ]:
custom_cfg = deepcopy(data_cfg)
custom_cfg['n_points'] = 512  # grille plus légère pour l'exploration rapide
custom_cfg['n_train'] = 4096
custom_cfg['n_val'] = 512
custom_cfg['batch_size'] = 32

# Ajustement fin de chaque intervalle physique
custom_cfg['train_ranges'] = deepcopy(custom_cfg['train_ranges_base'])
custom_cfg['train_ranges']['mf_CH4'] = [5e-6, 4e-5]
custom_cfg['train_ranges']['baseline1'] = [-5e-4, -2e-4]
custom_cfg['train_ranges']['baseline2'] = [-5e-8, -2e-8]
custom_cfg['train_ranges']['P'] = [350, 650]
custom_cfg['train_ranges']['T'] = [300.0, 320.0]

# Bruit plus agressif pour visualiser son impact
custom_cfg['noise']['train'] = {
    'std_add_range': [0.0, 0.03],
    'std_mult_range': [0.0, 0.02],
    'p_drift': 0.3,
    'drift_sigma_range': [12.0, 90.0],
    'drift_amp_range': [0.002, 0.06],
    'p_fringes': 0.5,
    'n_fringes_range': [1, 3],
    'fringe_freq_range': [0.2, 35.0],
    'fringe_amp_range': [0.001, 0.02],
    'p_spikes': 0.2,
    'spikes_count_range': [1, 4],
    'spike_amp_range': [0.001, 0.5],
    'spike_width_range': [1.0, 15.0],
    'clip': [0.0, 1.2],
}

pprint(custom_cfg['train_ranges'])

## Préparation des coefficients spectraux

On fournit explicitement le polynôme `poly_freq_CH4` et les transitions CH₄ utilisées par défaut. Rien n'empêche de remplacer ces coefficients par un autre fichier pour couvrir d'autres gaz.

In [ ]:
poly_freq_coeffs = [-2.3614803e-07, 1.2103413e-10, -3.1617856e-14]
transitions_dict = {'CH4': parse_csv_transitions("6;1;3085.861015;1.013E-19;0.06;0.078;219.9411;0.73;-0.00712;0.0;0.0221;0.96;0.584;1.12\n6;1;3085.832038;1.693E-19;0.0597;0.078;219.9451;0.73;-0.00712;0.0;0.0222;0.91;0.173;1.11\n6;1;3085.893769;1.011E-19;0.0602;0.078;219.9366;0.73;-0.00711;0.0;0.0184;1.14;-0.516;1.37\n6;1;3086.030985;1.659E-19;0.0595;0.078;219.9197;0.73;-0.00711;0.0;0.0193;1.17;-0.204;0.97\n6;1;3086.071879;1.000E-19;0.0585;0.078;219.9149;0.73;-0.00703;0.0;0.0232;1.09;-0.0689;0.82\n6;1;3086.085994;6.671E-20;0.055;0.078;219.9133;0.70;-0.00610;0.0;0.0300;0.54;0.00;0.0")}

# Normalisation globale utilisée par SpectraDataset
config.set_norm_params({name: tuple(values) for name, values in custom_cfg['train_ranges'].items()})

## Instanciation d'un `SpectraDataset` complet

Cette cellule construit deux jeux de données indépendants (train et validation) en appliquant toutes les modifications ci-dessus. Chaque argument peut être remplacé dynamiquement.

In [ ]:
def _to_tuple_ranges(mapping):
    return {k: tuple(v) for k, v in mapping.items()}

noise_train = {k: tuple(v) if isinstance(v, list) else v for k, v in custom_cfg['noise']['train'].items()}
noise_val = {k: tuple(v) if isinstance(v, list) else v for k, v in custom_cfg['noise']['val'].items()}

dataset_train = SpectraDataset(
    n_samples=custom_cfg['n_train'],
    num_points=custom_cfg['n_points'],
    poly_freq_CH4=poly_freq_coeffs,
    transitions_dict=transitions_dict,
    sample_ranges=_to_tuple_ranges(custom_cfg['train_ranges']),
    strict_check=True,
    with_noise=True,
    noise_profile=noise_train,
    freeze_noise=False,
)

dataset_val = SpectraDataset(
    n_samples=custom_cfg['n_val'],
    num_points=custom_cfg['n_points'],
    poly_freq_CH4=poly_freq_coeffs,
    transitions_dict=transitions_dict,
    sample_ranges=_to_tuple_ranges(custom_cfg['val_ranges']),
    strict_check=True,
    with_noise=True,
    noise_profile=noise_val,
    freeze_noise=True,
)

len(dataset_train), len(dataset_val)

## Visualisation interactive d'échantillons

Ce bloc montre comment produire des spectres propres/bruyants et visualiser les paramètres tirés. Les valeurs peuvent être fixées en remplaçant n'importe quelle plage par un intervalle dégénéré.

In [ ]:
def plot_example(idx=0):
    sample = dataset_train[idx]
    noisy = sample['noisy_spectra'].numpy()
    clean = sample['clean_spectra'].numpy()
    params = sample['params'].numpy()
    fig, ax = plt.subplots(figsize=(10, 4))
    ax.plot(noisy, label='bruité')
    ax.plot(clean, label='propre', alpha=0.7)
    ax.set_title(f'Echantillon {idx}')
    ax.set_xlabel('Indice de la grille')
    ax.set_ylabel('Transmission normalisée')
    ax.legend()
    fig.tight_layout()
    return fig, ax, params

plot_example(0)

## Balayage de paramètres ciblé

Le helper suivant fixe un paramètre à une valeur choisie et régénère instantanément un spectre afin de visualiser son impact. On peut faire varier simultanément plusieurs paramètres en passant un dictionnaire d'overrides.

In [ ]:
def generate_spectrum(overrides=None, *, with_noise=False):
    overrides = overrides or {}
    forced_ranges = _to_tuple_ranges(custom_cfg['train_ranges'])
    for key, value in overrides.items():
        forced_ranges[key] = (value, value)

    dataset = SpectraDataset(
        n_samples=1,
        num_points=custom_cfg['n_points'],
        poly_freq_CH4=poly_freq_coeffs,
        transitions_dict=transitions_dict,
        sample_ranges=forced_ranges,
        strict_check=False,
        with_noise=with_noise,
        noise_profile=noise_train,
        freeze_noise=True,
    )
    sample = dataset[0]
    return sample['noisy_spectra'].numpy(), sample['clean_spectra'].numpy()

def sweep_parameter(name, values, *, with_noise=False):
    fig, ax = plt.subplots(figsize=(10, 4))
    for value in values:
        noisy, clean = generate_spectrum({name: value}, with_noise=with_noise)
        ax.plot(clean, label=f'{name}={value}', alpha=0.7)
    ax.set_title(f'Impact de {name}')
    ax.set_xlabel('Indice de la grille')
    ax.set_ylabel('Transmission normalisée')
    ax.legend()
    fig.tight_layout()
    return fig, ax

# Exemple : variation de la concentration de CH4
sweep_parameter('mf_CH4', [5e-6, 1e-5, 2e-5, 3.5e-5])